In [8]:
import pandas as pd
from database import select_dataset
import numpy as np

from graphmodels.graphmodel import GraphModel, converte_to_format, concatenate_graph_specs
from graphmodels.validation import score_model, plot_diagnostic, plot_baseline_vs_computation
from graphmodels.data_utils import data_dict_from_df_tables, get_X_y_from_data
from models.Sarah.model_EW import EW_models, MWU_model


In [10]:
df_tables = ['aquastat', 'evapotranspiration', 'cropcalendar', 'efficientwater', 'worldbank']

data_dict = data_dict_from_df_tables(df_tables)
data_dict['LWU'] = 0

In [13]:
# FIND WAY TO WRAP THIS CLEANLY CAUSE IT's A MESS !

aquastat = select_dataset('aquastat').dropna().drop(columns=['Area'])
aquastat = aquastat[~aquastat.ISO.isin(['USA', 'CHN', 'IND', 'RUS'])] # Not parsed !
aquastat = aquastat[aquastat.Variable != 'CI']
aquastat['Year'] = aquastat['Year'].astype(int)

evapotranspiration = select_dataset('evapotranspiration').dropna().drop_duplicates()
cropcalendar = select_dataset('cropcalendar').drop(columns=['Year'])
efficientwater = select_dataset('efficientwater')
wb_data = select_dataset('worldbank')
wb_data['Year'] = wb_data['Year'].astype(int)

MWU_df = pd.concat([aquastat, evapotranspiration, cropcalendar, efficientwater, wb_data])

In [14]:
aquastat.Year.unique()

array([1992, 1997, 2002, 2007, 2012, 2016, 2017, 1998, 1990, 1995, 2000,
       2006, 2001, 2005, 2011, 1996, 2010, 2003, 2008, 1989, 2015, 1993,
       1994, 1999, 2004, 1991, 2014, 2013, 2009, 1988, 1987])

In [15]:
from graphmodels.utils import df_to_data_dict

In [16]:
itemized = ['Kc', 'ICA']
data_dict = df_to_data_dict(MWU_df, itemized)
data_dict['LWU'] = 0

# Model

In [17]:
from graphmodels.graphmodel import GraphModel, converte_to_format, concatenate_graph_specs
import numpy as np
from graphmodels.validation import score_model, plot_diagnostic
from models.Sarah.model_EW import EW_models, MWU_model
from graphmodels.utils import get_X_y_from_data

In [18]:
model = EW_models['IWW_model']
X, y = get_X_y_from_data(model, data_dict)
res = model.run(X)

In [19]:
for model_name, model in EW_models.items():
    print(model_name, ': ')
    X, y = get_X_y_from_data(model, data_dict)
    res = model.run(X)
    print(score_model(model, X, y))

IWW_model : 
IWR
IWW
AWU
         r2  correlation        rmse Variable
0  0.701133     0.842156   95.126928      IWR
1  0.824554     0.908953  125.996127      IWW
2  0.558706     0.803052  276.259000      AWU
EW1_partial_model : 
TWW
EW1
         r2  correlation         rmse Variable
0  0.986726     0.993346     6.739096      TWW
1  0.678466     0.853669  3689.291330      EW1
EW2_partial_model : 
TFA
EW2
         r2  correlation          rmse Variable
0  0.987030     0.994251  16931.898148      TFA
1  0.939918     0.973379     71.957636      EW2
EW1_model : 
IWR
IWW
AWU
MWU
TWW
EW1
         r2  correlation        rmse Variable
0  0.701133     0.842156   95.126928      IWR
1  0.824554     0.908953  125.996127      IWW
2  0.558706     0.803052  276.259000      AWU
3  0.806460     0.901572    1.200052      MWU
4  0.659551     0.839646  304.313181      TWW
5  0.279501     0.747804   83.731558      EW1
EW2_model : 
IWR
IWW
AWU
MWU
TFA
EW2
         r2  correlation          rmse Variable
0  0

In [20]:
X, y = get_X_y_from_data(EW_models['EW1_model'], data_dict)
plot_diagnostic(EW_models['EW1_model'], X, y, 'EW1', hover_data=['ISO'])